In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install trl
!pip install rouge_score
!pip install langchain_community

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    statu

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import warnings
import pandas as pd
from transformers import pipeline
from langchain.chains import ConversationChain, LLMChain
from langchain.memory import ConversationBufferWindowMemory, ConversationSummaryBufferMemory
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from rouge_score import rouge_scorer
import re
import warnings
warnings.filterwarnings('ignore')

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [ ]:
model_name = "ybelkada/falcon-7b-sharded-bf16"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
PEFT_MODEL = "AgeNtX071/Pefted"

config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)

peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

In [ ]:
df = pd.read_csv('/content/generated_texts.csv')
test = df['Generated Text'].tolist()

# Calculating Rouge

In [ ]:
def res(query):
    system_prompt = """Answer the following question truthfully.
    If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
    If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'."""

    user_prompt = f"""<HUMAN>: {query}
    <ASSISTANT>: """

    final_prompt = system_prompt + "\n" + user_prompt
    device = "cuda:0"

    peft_encoding = peft_tokenizer(final_prompt, return_tensors="pt").to(device)
    peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=100, pad_token_id = peft_tokenizer.eos_token_id, \
                                                                                                                      eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                      temperature=0.1, top_p=0.6, repetition_penalty=1.3, num_return_sequences=1,))
    peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)

    return peft_text_output[234:]  # Returning the generated response


In [ ]:

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

# Initialize variables to accumulate F-measure scores
total_f_measure_rouge1 = 0
total_f_measure_rougeL = 0
tota_prec1=0
tota_recall = 0
tota_recallL = 0
for i in range(70):
  text = test[i]
  query = text.split('\n', 1)[0]  # Extract everything before the first newline
  reference_match = re.search(r'<ASSISTANT>:(.*)', text)
  if reference_match:
        reference_response = reference_match.group(1).strip()  # Extract what's after "<ASSISTANT>:" and trim whitespace
  else:
        print(f"No reference response found in row {i + 1}. Skipping...")
        continue
  reference_response = re.sub(r'<(\/)?[A-Z]+>: ', '', reference_response)
  print(query)
  print(reference_response)

    # Generate response using your function
  generated_response = res(query)
  print('\n')

    # Remove <HUMAN> and <ASSISTANT> tags
  generated_response = re.sub(r'<(\/)?[A-Z]+>: ', '', generated_response)
  reference_response = re.sub(r'<(\/)?[A-Z]+>: ', '', reference_response)

  print(generated_response)

    # Calculate ROUGE scores
  rouge_scores = scorer.score(reference_response, generated_response)

    # Accumulate F-measure scores
  tota_prec1 += rouge_scores['rouge1'].precision
  tota_recall += rouge_scores['rouge1'].recall
  tota_recallL+= rouge_scores['rougeL'].recall
  total_f_measure_rouge1 += rouge_scores['rouge1'].fmeasure
  total_f_measure_rougeL += rouge_scores['rougeL'].fmeasure

# Calculate average F-measure scores
avg_f_measure_rouge1 = total_f_measure_rouge1/70
avg_f_measure_rougeL = total_f_measure_rougeL/70
avg_rougeL =tota_recallL/70
avg_rouge1 =tota_recall/70

# Print average F-measure scores
print(f"Average ROUGE-1 F-measure: {avg_f_measure_rouge1}")
print(f"Average ROUGE-L F-measure: {avg_f_measure_rougeL}")



# Comparision With Baseline

In [ ]:
# Function to generate responses from both original model and PEFT model and compare their answers.
def generate_answer(query):
  system_prompt = """Answer the following question truthfully.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
  If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'."""

  user_prompt = f"""<HUMAN>: {query}
  <ASSISTANT>: """

  final_prompt = system_prompt + "\n" + user_prompt

  device = "cuda:0"
  dashline = "-".join("" for i in range(50))

  encoding = tokenizer(final_prompt, return_tensors="pt").to(device)
  outputs = model.generate(input_ids=encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = tokenizer.eos_token_id, \
                                                                                                                     eos_token_id = tokenizer.eos_token_id, attention_mask = encoding.attention_mask, \
                                                                                                                     temperature=0.2, top_p=0.6, repetition_penalty=1.3, num_return_sequences=1,))
  text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

  print(dashline)
  print(f'ORIGINAL MODEL RESPONSE:\n{text_output[234:]}')
  print(dashline)

  peft_encoding = peft_tokenizer(final_prompt, return_tensors="pt").to(device)
  peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = peft_tokenizer.eos_token_id, \
                                                                                                                     eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                     temperature=0.2, top_p=0.6, repetition_penalty=1.3, num_return_sequences=1,))
  peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)

  print(f'PEFT MODEL RESPONSE:\n{peft_text_output[234:]}')
  print(dashline)

# Chat-Bot Simulation

In [ ]:
pipe = pipeline(task="text-generation",
                model=peft_model,
                tokenizer=peft_tokenizer,
                max_new_tokens=512,
                num_beams=10,
                early_stopping=True,
                no_repeat_ngram_size=2)

llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
template = """
Answer the following question truthfully.
If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'.

Previous conversation:
{chat_history}
<HUMAN> : {query}
<ASSISTANT>:
"""

prompt = PromptTemplate.from_template(template)
memory=ConversationBufferMemory(memory_key="chat_history")
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [ ]:
conv_res = conversation({"query":"My name is uday, Im feeling very low"})
print(f"Answer: {conv_res['text']}")

conv_res = conversation({"query":"what was my name i had said?"})
print(f"Answer: {conv_res['text']}")